In [5]:
import json
import numpy as np
import pandas as pd
import nlbayes
from nlbayes.utils import read_network_json, get_evidence_dict

In [2]:
# download files: 
#   - This differential expression table was generated using the GEO2R tool, contrasting the E2F3 treated
#     samples against the control samples. At GEO2R, we need to select the `Gene.ID` column that contains
#     Entrez (NCBI) gene ids.
#     url: https://umbibio.math.umb.edu/nlbayes/assets/data/experiments/GSE3151.E2F3.top.table.tsv
#   - Since the experiment above was performed on mammary epithelial cell cultures, we may choose a breast
#     tissue specific network. 
#     url: https://umbibio.math.umb.edu/nlbayes/assets/data/networks/gtex_chip/homo_sapiens/tissue_specific/breast.rels.json

In [6]:
network = read_network_json("breast.rels.json")

In [8]:
evidence = pd.read_csv("GSE3151.E2F3.top.table.tsv", sep='\t')
evidence = get_evidence_dict(
        evidence,
        logfc_threshold=1,      # limit DE genes by log2fold-change (logFC)
        pval_threshold=0.001,   # limit DE genes by p-value
        network=network,        # optional. If provided, genes will be selected
                                # only if present in network's set of targets
    )
print(f"\nSelected {len(evidence)} DE genes")

Using column `Gene.ID` as gene
Using column `adj.P.Val` as pval
Using column `logFC` as logfc

Selected 255 DE genes


In [9]:
model = nlbayes.ModelORNOR(network, evidence, uniform_t = False, n_graphs=5)


In [10]:
model.sample_posterior(N=2000, gr_level=1.2, burnin=True)


Initializing model burn-in ...



100%|██████████| 20/20 [00:23<00:00,  1.16s/it]


Converged after 20 samples
Max Gelman-Rubin statistic is 2.0185805713786285 (target was 5.0 )
Burn-in complete ...



100%|██████████| 165/165 [03:20<00:00,  1.21s/it]

Converged after 165 samples
Max Gelman-Rubin statistic is 1.1995180616707848 (target was 1.2 )


In [11]:
df = model.inference_posterior_df()
df

,TF_id,X,T
rank,,,
1,3169,1.000000,0.840077
2,1869,1.000000,0.859877
3,6256,0.998485,0.786958
4,367,0.987879,0.784177
5,5914,0.983333,0.768444
...,...,...,...
745,7546,0.003030,0.485636
746,2972,0.001515,0.478635
747,57491,0.001515,0.501388


In [ ]:
# we would like to convert NCBI ids to gene symbols
from biomart import BiomartServer

# Connect to the BioMart server
server = BiomartServer("http://www.ensembl.org/biomart")

# Set the dataset to use
dataset = server.datasets["hsapiens_gene_ensembl"]

# Define the attributes to retrieve
attributes = ["entrezgene_id", "external_gene_name"]

# Define the filters to use (e.g. gene ids to convert)
filters = {"entrezgene_id": df['TF_id'].to_list()}


In [ ]:

# Perform the query
response = dataset.search({'attributes':attributes, 'filters':filters})

annotation = {}
for record in response.iter_lines():
    record = record.decode("utf-8").strip().split("\t")
    if len(record) == 2:
        annotation[record[0]] = record[1]
    else:
        annotation[record[0]] = ''

In [38]:

# Process the results
df['TF_symbol'] = df['TF_id'].map(annotation)
df.iloc[:,[0, 3, 1, 2]].head(50)


,TF_id,TF_symbol,X,T
rank,,,,
1,3169,FOXA1,1.000000,0.840077
2,1869,E2F1,1.000000,0.859877
3,6256,RXRA,0.998485,0.786958
4,367,AR,0.987879,0.784177
5,5914,RARA,0.983333,0.768444
6,4613,MYCN,0.930303,0.757959
7,429,ASCL1,0.843939,0.707753
8,8320,EOMES,0.686364,0.650892
9,2625,GATA3,0.678788,0.660604
